In [ ]:
import json
import os
import yaml
from alchemiscale import AlchemiscaleClient, Scope
from openff.units import unit
from openff.toolkit import Molecule
from perses.protocols import NonEquilibriumCyclingProtocol
from gufe import ChemicalSystem, SmallMoleculeComponent, ProteinComponent, SolventComponent, AlchemicalNetwork, tokenization
from gufe.mapping.ligandatommapping import LigandAtomMapping
from gufe.transformations import Transformation

In [ ]:
# Helper functions
def get_molecule_from_name(name: str, mol_list: list[Molecule]):
    """Get the OpenFF molecule from a list that matches the specified name.
    
    Returns the molecule if found, else None.    
    """
    for molecule in mol_list:
        if molecule.name == name:
            matched_mol = molecule
            break
    else:
        matched_mol = None
    
    return matched_mol

In [ ]:
# Load target/protein
protein_comp = ProteinComponent.from_pdb_file("tyk2_protein.pdb", name="Tyk2")
# Load ligands to match with edges
ligands = Molecule.from_file("tyk2_ligands.sdf")
# Solvent component
solvent_comp = SolventComponent(
    ion_concentration=0.15*unit.molar,
    positive_ion="Na",
    negative_ion="Cl"
)    

In [ ]:
# Extract edges information from YAML file -- Build ChemicalSystems and AlchemicalNetworks
with open("tyk2_edges.yml") as edges_file:
    edges_source = yaml.safe_load(edges_file)
    
# Load default settings for protocol
settings = NonEquilibriumCyclingProtocol.default_settings()
# Use latest (to date) openff-2.1.0 instead of default openff-2.0.0
settings.forcefield_settings.small_molecule_forcefield = "openff-2.1.0"
neq_protocol = NonEquilibriumCyclingProtocol(settings=settings)

solvent_transformations = []
complex_transformations = []
alchemical_nodes = []
for edge_name, data in edges_source["edges"].items():
    ligand_a_name = data["ligand_a"]
    ligand_b_name = data["ligand_b"]
    atom_mapping = data["atom mapping"]
    # Get OFF molecule and build SmallMoleculeComponent
    ligand_a = get_molecule_from_name(ligand_a_name, ligands)
    ligand_b = get_molecule_from_name(ligand_b_name, ligands)
    small_mol_a = SmallMoleculeComponent.from_openff(ligand_a)
    small_mol_b = SmallMoleculeComponent.from_openff(ligand_b)
    # Build state dictionaries -- useful for instantiating the ChemicalSystems
    state_a_complex = {"protein": protein_comp, "ligand": small_mol_a, "solvent": solvent_comp}
    state_b_complex = {"protein": protein_comp, "ligand": small_mol_b, "solvent": solvent_comp}
    state_a_solvent = {"ligand": small_mol_a, "solvent": solvent_comp}
    state_b_solvent = {"ligand": small_mol_b, "solvent": solvent_comp}
    # Build ChemicalSystem from state dictionaries
    system_a_complex = ChemicalSystem(components=state_a_complex)
    system_b_complex = ChemicalSystem(components=state_b_complex)
    system_a_solvent = ChemicalSystem(components=state_a_solvent)
    system_b_solvent = ChemicalSystem(components=state_b_solvent)
    # Add to alchemical transformation nodes
    alchemical_nodes.extend([system_a_complex, system_a_solvent, system_b_complex, system_b_solvent])
    # Fill transformations iterables
    solvent_transformations.append(
        Transformation(
            stateA=system_a_solvent,
            stateB=system_b_solvent,
            mapping={"ligand": atom_mapping},
            protocol=neq_protocol,
        )
    )
    complex_transformations.append(
        Transformation(
            stateA=system_a_complex,
            stateB=system_b_complex,
            mapping={"ligand": atom_mapping},
            protocol=neq_protocol,
        )
    )

# Create alchemical network joining all the transformations/edges together
network = AlchemicalNetwork(
    edges=(solvent_transformations + complex_transformations),
    nodes=alchemical_nodes,
    name="tyk2_plbenchmarks_off2.0",
)
# print generated network and serialize
print("generated network: ", network)
with open('network.json', 'w') as f:
    json.dump(network.to_dict(), f, cls=tokenization.JSON_HANDLER.encoder)

To create and actions tasks we need to have an active running alchemiscale client session

In [ ]:
# Set alchemiscale client up
user_id = os.environ['ALCHEMISCALE_ID']
user_key = os.environ['ALCHEMISCALE_KEY']
asc =  AlchemiscaleClient('https://api.alchemiscale.org', user_id, user_key)
asc.list_scopes()  # print available scopes for these credentials
# Get an scope we want to use to action tasks
scope = Scope('choderalab', 'plbenchmarks', 'tyk2_off2_1')

In [ ]:
asc.get_scoped_key(network, scope)

In [ ]:
# Create and actions tasks -- example 100 runs per edge for NEQ cycling
network_sk = asc.create_network(network, scope)
whole_tasks = []
for transform_sk in asc.get_network_transformations(network_sk):
    tasks = asc.create_tasks(transform_sk, count=100)
    whole_tasks.extend(tasks)
asc.action_tasks(whole_tasks, network_sk)